In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

import numpy as np

# 自定义门

标准门（如Pauli门和CNOT门）在`cirq.ops`中定义， 具体参见下一节 操作Operation 。

要在电路中使用非标准门的酉矩阵，可以按照本指南创建自定义门。


## 一般模式
在Cirq中，门是类。要定义自定义门，我们需要继承自一个基础门类并定义一些方法。  

一般模式是：

1. 继承自`cirq.Gate`。
2. 定义`_num_qubits_`或`_qid_shape_`方法之一。
3. 定义`_unitary_`或`_decompose_`方法之一。

> 注意：以一个或多个下划线开头和结尾的方法是魔法方法，它们被Cirq的协议或内置Python函数使用。关于魔法方法的更多信息将在本指南的末尾提供。

接下来将通过以下示例演示这些模式。

## 酉矩阵创建
可以通过以下方式从酉矩阵创建自定义Cirq门。这里，我们定义一个对应于酉矩阵的门。

$$
H = \frac{1}{\sqrt{2}} \begin{pmatrix}
1 & 1 \\
-1 & 1
\end{pmatrix}
$$

In [2]:
class MyGate(cirq.Gate):
    def __init__(self):
        super(MyGate, self)

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [1.0, 1.0],
            [-1.0, 1.0]
        ]) /np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "MyGate"

my_gate = MyGate()

在这个示例中，`_num_qubits_`方法告诉Cirq这个门作用于单个量子比特，`_unitary_`方法定义了门的酉矩阵。`_circuit_diagram_info_`方法告诉Cirq如何在电路中显示这个门，如下所示。

一旦我们定义了这个门，它可以像Cirq中的任何标准门一样使用。

In [3]:
"""在电路中使用这个门"""
circ = cirq.Circuit(
    my_gate.on(cirq.LineQubit(0))
)
print("包含自定义门的电路：")
print(circ)

包含自定义门的电路：
0: ───MyGate───


当我们打印电路时，我们会看到在`_circuit_diagram_info_`方法中指定的符号。

带有自定义门的电路可以像带有标准门的电路一样进行模拟。

In [4]:
"""用这个门模拟一个量子电路"""
sim = cirq.Simulator()
res = sim.simulate(circ)
print("模拟结果：")
print(res)

模拟结果：
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: 0.707|0⟩ - 0.707|1⟩

phase:
output vector: |⟩


In [8]:
"""定义一个两个量子比特的门"""
class TwoQubitGate(cirq.Gate):
    def __init__(self):
        super(TwoQubitGate, self)

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([
            [1.0, -1.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 1.0],
            [1.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, -1.0]
        ])/np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "顶线", "底线"

two_qubit_gate = TwoQubitGate()

"""在电路中使用这个门"""
circ = cirq.Circuit(
    two_qubit_gate.on(*cirq.LineQubit.range(2))
)

print("Circuit with custom two-qubit gate:")
print(circ)

Circuit with custom two-qubit gate:
0: ───顶线───
      │
1: ───底线───


这个电路也可以按预期方式进行模拟，不负所望😄

## 带参数的自定义门

自定义门可以使用参数进行定义和使用。例如，定义一个带参数的门：

$$
R(\theta) =  \begin{pmatrix}
cos\theta & sin\theta \\
sin\theta & -cos\theta
\end{pmatrix}
$$

In [12]:
"""定义一个参数化门"""
class RotationGate(cirq.Gate):
    def __init__(self, theta):
        super(RotationGate, self)
        self.theta = theta
    
    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [np.cos(self.theta), np.sin(self.theta)],
            [np.sin(self.theta), -np.cos(self.theta)]
        ])/ np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return f"R({self.theta})"

""""在电路中使用这个门"""
circ = cirq.Circuit(
    RotationGate(theta=0.1).on(cirq.LineQubit(0))
)

print("Circuit with custom parameterized gate:")
print(circ)


Circuit with custom parameterized gate:
0: ───R(0.1)───


## 从已知分解定义
自定义门也可以从已知的门分解定义。这在电路中重复出现的门组或已知门的标准分解为原始门时非常有用。

下面我们展示一个自定义交换门的示例，该门从三个CNOT门的已知分解定义。


In [13]:
class MySwap(cirq.Gate):
    def __init__(self):
        super(MySwap, self)

    def _num_qubits_(self):
        return 2
    
    def _decompose_(self, qubits):
        a, b = qubits
        yield cirq.CNOT(a, b)
        yield cirq.CNOT(b, a)
        yield cirq.CNOT(a, b)
    
    def _circuit_diagram_info_(self, args):
        return ["CustumSwap"] * self._num_qubits_()

my_swap = MySwap()

`_decompose_`方法生成实现自定义门的操作。（也可以返回操作列表而不是生成器。）

当我们在电路中使用这个门时，分解中的各个门不会出现在电路中。相反，电路中会显示`_circuit_diagram_info_`。如前所述，这对于在比单个（原始）门更高的层次上解释电路非常有用。

In [14]:
""" 在电路中使用这个门 """
qreg = cirq.LineQubit.range(2)
circ = cirq.Circuit(
    cirq.X(qreg[0]),
    my_swap.on(*qreg)
)
print("电路中包含自定义交换门：")
print(circ)


电路中包含自定义交换门：
0: ───X───CustumSwap───
          │
1: ───────CustumSwap───


我们可以模拟这个电路并验证它是否真的交换了量子比特。

In [16]:
""" 模拟这个电路 """
sim.simulate(circ)

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: |01⟩

phase:
output vector: |⟩

## 更多关于魔法方法和协议
(本部分机翻，不通顺地方，参见原文 [custom gates](https://quantumai.google/cirq/build/custom_gates))  

如前所述，诸如`_unitary_`之类的方法被称为“魔法方法”。Cirq的大部分功能依赖于“魔法方法”，这些方法以一个或两个下划线开头，并由Cirq的协议或内置Python方法使用。例如，Python将`cirq.Z**0.25`翻译为`cirq.Z.__pow__(0.25)`。其他用途特定于Cirq，并在protocols子目录中找到。它们定义如下。

至少，你需要定义`_num_qubits_`或`_qid_shape_`魔法方法之一，以定义门使用的量子比特（或qudits）数量。

### 标准Python魔法方法
Python中有许多标准的魔法方法。以下是Cirq中使用的一些最重要的方法：

- `__str__` 用于用户友好的字符串输出，`__repr__` 是Python友好的字符串输出，意味着`eval(repr(y))==y`应该始终为真。
- `__eq__` 和 `__hash__` 定义对象是否相等。你也可以使用`cirq.value.value_equality`来比较具有少量子值的对象是否相等。
- 算术函数如 `__pow__`、`__mul__`、`__add__` 分别定义 `**`、`*` 和 `+` 的操作。

### cirq.num_qubits 和 _num_qubits_ 方法
一个门必须实现`_num_qubits_`（或`_qid_shape_`）方法。此方法返回一个整数，并由`cirq.num_qubits`用于确定此门作用于多少个量子比特。

### cirq.qid_shape 和 _qid_shape_ 方法
一个qudit门或操作必须实现`_qid_shape_`方法，该方法返回一个整数元组。此方法用于确定门或操作作用于多少个qudits以及每个qudit的维度。如果仅实现`_num_qubits_`方法，则假定对象仅作用于量子比特。调用者可以通过调用`cirq.qid_shape`来查询对象的qid形状。有关更多信息，请参阅qudit文档。

### cirq.unitary 和 _unitary_ 方法
当一个对象可以用酉矩阵描述时，它可以通过实现`_unitary_(self) -> np.ndarray`方法来公开该酉矩阵。调用者可以通过调用`cirq.unitary`来查询对象是否具有酉矩阵。`_unitary_`方法也可以返回`NotImplemented`，在这种情况下，`cirq.unitary`的行为就像该方法未实现一样。

### cirq.decompose 和 _decompose_ 方法
操作和门可以通过实现返回其他操作的`_decompose_`方法来定义。操作实现`_decompose_(self)`，而门实现`_decompose_(self, qubits)`（因为门事先不知道它们的量子比特）。

`_decompose_`方法的主要要求是：
- 不要创建循环。`cirq.decompose`方法将迭代分解，直到找到满足保留谓词的值。循环会导致它进入无限循环。
- 朝向Cirq定义的操作，因为这些操作具有良好的分解方法，最终在单量子比特和双量子比特门中终止。这些门可以被模拟器、优化器和其他代码理解。
- 重要的是功能等效性。不要担心保持在特定的门集合内或达到特定的门集合；预测调用者的需求太难了。门集合感知的分解是有用的，但这不是实现该功能的协议。相反，使用transformer API中可用的功能。

例如，`cirq.CCZ`分解为一系列`cirq.CNOT`和`cirq.T`操作。这允许不理解三量子比特操作的代码通过分解它们来处理`cirq.CCZ`；通过将其分解为它们理解的操作。另一个例子是`cirq.TOFFOLI`分解为一个`cirq.H`，然后是一个`cirq.CCZ`，然后是一个`cirq.H`。虽然输出包含一个三量子比特操作（CCZ），但该操作可以分解为双量子比特和单量子比特操作。因此，不理解三量子比特操作的代码可以通过分解它们来处理Toffolis，然后分解初始分解产生的CCZ。

一般来说，分解感知代码消耗操作，期望递归分解未知操作，直到代码遇到它理解的操作或遇到无法进一步分解的死胡同。`cirq.decompose`方法实现了执行这种递归分解的逻辑。调用者指定一个保留谓词，并可选地指定拦截和回退分解器，然后`cirq.decompose`将反复分解给定的操作，直到操作满足给定的保留谓词。如果`cirq.decompose`遇到死胡同，它会引发错误。

Cirq不对“目标门集合”的分解方向做任何保证。`cirq.decompose`不是一个方法。Cirq中的分解趋向于X、Y、Z、CZ、PhasedX、指定矩阵门等。但这个集合会因版本而异，因此分解的消费者需要寻找门的通用属性，例如“具有酉矩阵的双量子比特门”，而不是特定的门类型如CZ门。

### cirq.inverse 和 __pow__
当门和操作实现了`__pow__`方法并且对于指数-1返回一个结果（而不是`NotImplemented`）时，它们被认为是可逆的。这个逆可以直接通过`value**-1`访问，或者通过实用方法`cirq.inverse(value)`访问。如果你确定`value`有逆，使用`value**-1`比使用`cirq.inverse(value)`更方便。`cirq.inverse`适用于你不确定`value`是否可逆，或者`value`可能是一系列可逆操作的情况。

`cirq.inverse`有一个默认参数，当`value`不可逆时用作回退。例如，`cirq.inverse(value, default=None)`返回`value`的逆，否则返回`None`（如果`value`不可逆）。如果没有指定默认值且`value`不可逆，则会引发`TypeError`。

当你给`cirq.inverse`一个列表或任何其他类型的可迭代对象时，它将返回一个操作序列，这些操作（按顺序运行）将撤销原始序列的操作（按顺序运行）。基本上，列表中的项目被单独反转并以相反的顺序返回。例如，表达式`cirq.inverse([cirq.S(b), cirq.CNOT(a, b)])`将返回元组`(cirq.CNOT(a, b), cirq.S(b)**-1)`。

门和操作也可以为除-1以外的指数从它们的`__pow__`方法返回值。Cirq经常使用这种模式。例如，可以通过将`cirq.X`提升到0.5来创建X门的平方根：
